<a href="https://colab.research.google.com/github/MiyoBran/Alura-ONE-G9/blob/main/formacion-Aprendiendo-a-hacer-ETL-G9-ONE/02-pandas-e-s-diferentes-formatos-archivo/ETL_02_Pandas_I_O_HTML_XML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Aula 4 - Leyendo páginas en HTML e XML**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##**Importando páginas web**

In [2]:
import pandas as pd

In [3]:
datos_html = pd.read_html('/content/drive/MyDrive/Pandas/peliculas_wikipedia.html')

In [4]:
#Nuestra extraccion no nos muestra el Dataframe buscado
datos_html

[       0                      1
 0   1998             100 Movies
 1   1999              100 Stars
 2   2000             100 Laughs
 3   2001            100 Thrills
 4   2002           100 Passions
 5   2003  100 Heroes & Villains
 6   2004              100 Songs
 7   2005       100 Movie Quotes
 8   2005              25 Scores
 9   2006             100 Cheers
 10  2006            25 Musicals
 11  2007   100 Movies (Updated)
 12  2008        AFI's 10 Top 10
 13   vte                    vte,
                             Film  Release year              Director  \
 0                   Citizen Kane          1941          Orson Welles   
 1                     Casablanca          1942        Michael Curtiz   
 2                  The Godfather          1972  Francis Ford Coppola   
 3             Gone with the Wind          1939        Victor Fleming   
 4             Lawrence of Arabia          1962            David Lean   
 ..                           ...           ...                   

In [5]:
# Se extrajo una lista que tiene dentro 3 objetos , en este caso las tablas
type(datos_html)

list

In [6]:
len(datos_html)

3

In [7]:
# Al visualizar la pagina , e inspeccionar, vemos que es la segunda tabla
# (posicion 1)
top_peliculas = datos_html[1]

In [8]:
top_peliculas

,Film,Release year,Director,Production companies,Rank
0,Citizen Kane,1941,Orson Welles,RKO Radio Pictures,1
1,Casablanca,1942,Michael Curtiz,Warner Bros. Pictures,2
2,The Godfather,1972,Francis Ford Coppola,"Paramount Pictures, Alfran Productions",3
3,Gone with the Wind,1939,Victor Fleming,Selznick International Pictures,4
4,Lawrence of Arabia,1962,David Lean,Horizon Pictures,5
...,...,...,...,...,...
95,The Searchers,1956,John Ford,C. V. Whitney Pictures,96
96,Bringing Up Baby,1938,Howard Hawks,RKO Radio Pictures,97
97,Unforgiven,1992,Clint Eastwood,The Malpaso Company,98
98,Guess Who's Coming to Dinner,1967,Stanley Kramer,Columbia Pictures,99


### 🌐 Web Scraping Básico: Extracción de Tablas HTML

**Concepto de Negocio:**
Muchas instituciones publican datos valiosos directamente en el cuerpo de sus páginas web (HTML) sin ofrecer un botón de descarga. Pandas puede actuar como un "raspador" (scraper) leyendo las etiquetas `<table>` del código fuente.

**Comportamiento de la API (`read_html`):**
A diferencia de otros métodos, `read_html` **siempre devuelve una lista** de DataFrames, ya que asume que puede haber múltiples tablas en la página.

**Estrategias de Extracción:**
1. **Por Índice (Frágil):** Extraer usando la posición de la tabla en la lista (ej. `lista[1]`). Si la página web cambia su diseño, el código se rompe.
2. **Por Coincidencia (Robusto):** Usar el parámetro `match='PalabraClave'`. Pandas buscará específicamente la tabla que contenga esa columna o texto, ignorando el orden visual de la página.

In [9]:
import pandas as pd

# 1. Definición de Rutas (Tu archivo guardado)
# tambien podriamos haber pasado el URL directamente en este punto , ya que
# read_html nos permite hacer la lectura de datos de una web publicada y pandas
# la descarga en tiempo real
RUTA_HTML = '/content/drive/MyDrive/Pandas/peliculas_wikipedia.html'

# -------------------------------------------------------------------
# Método 1: Exploración General (El enfoque del curso)
# -------------------------------------------------------------------
todas_las_tablas = pd.read_html(RUTA_HTML)
print(f"🔍 Exploración: Se encontraron {len(todas_las_tablas)} tablas en la página.")

# Extracción por índice duro (Segunda tabla)
df_por_indice = todas_las_tablas[1]


# -------------------------------------------------------------------
# Método 2: Extracción Robusta (El enfoque Senior)
# -------------------------------------------------------------------
# Le pedimos que extraiga SOLO la tabla que contenga la palabra 'Director'
# Como read_html devuelve una lista, tomamos el elemento [0] de esa lista filtrada
df_peliculas = pd.read_html(RUTA_HTML, match='Director')[0]

print("✅ Extracción HTML exitosa (Método Robusto).")
display(df_peliculas.head(3))

🔍 Exploración: Se encontraron 3 tablas en la página.
✅ Extracción HTML exitosa (Método Robusto).


,Film,Release year,Director,Production companies,Rank
0,Citizen Kane,1941,Orson Welles,RKO Radio Pictures,1
1,Casablanca,1942,Michael Curtiz,Warner Bros. Pictures,2
2,The Godfather,1972,Francis Ford Coppola,"Paramount Pictures, Alfran Productions",3


### 🕸️ Anatomía de la Web: El DOM y las Tablas HTML

"Cuando le pasas una URL a pd.read_html(), Pandas ignora completamente los párrafos (`<p>`), los títulos "(`<h1>`)" o las imágenes (`<img>`). Su algoritmo está programado exclusivamente para cazar la etiqueta `<table>` y desarmar su contenido."

**Concepto de Negocio (Web Scraping):**
Para extraer datos de internet de forma automatizada, debemos comprender la estructura jerárquica del lenguaje HTML (conocida como el DOM). Los datos no están flotando en la pantalla; están encapsulados en etiquetas específicas.

**El Algoritmo de Traducción de Pandas:**
La función `pd.read_html()` escanea el código fuente de la página ignorando elementos estéticos y centrándose exclusivamente en la estructura tabular:
1. `<table>`: Identifica el inicio de un potencial DataFrame.
2. `<tr>` *(Table Row)*: Lo traduce como un nuevo registro (Fila).
3. `<th>` *(Table Header)*: Lo traduce como el encabezado del DataFrame (Columnas).
4. `<td>` *(Table Data)*: Lo traduce como el valor individual de una celda en la intersección de la fila y columna.

### 🚧 Limitaciones de `read_html` y la Web Dinámica

**El Problema de Negocio (Web Moderna y Seguridad):**
La función `pd.read_html()` es estática y ciega. Solo puede leer el código fuente original que el servidor envía en el primer milisegundo. Sin embargo, las aplicaciones web modernas (como bancos o e-commerce) utilizan JavaScript para dibujar los datos en la pantalla *después* de cargar la página (Client-Side Rendering), o utilizan etiquetas modernas como `<div class="tabla">` en lugar de la clásica `<table>`. Además, los firewalls (como Cloudflare) bloquean las peticiones automatizadas simples.



**Matriz de Escalamiento (Plan de Contingencia):**
Cuando Pandas falla al encontrar la tabla, un Ingeniero de Datos aplica uno de estos tres caminos lógicos:

1. **Intercepción de APIs Ocultas (El más eficiente):** En lugar de "raspar" la pantalla, usamos la pestaña *Network* (F12 en Chrome) para descubrir de dónde saca la página web sus datos. Generalmente, descubriremos que la página consume un archivo JSON oculto. Tomamos esa URL y usamos `requests` + `json_normalize`.
2. **Automatización de Navegadores (Para alta seguridad/JS):** Utilizamos librerías como **Selenium** o **Playwright**. Estas herramientas abren un navegador real (invisible), esperan a que el JavaScript dibuje los datos en pantalla y superan barreras de seguridad básicas antes de extraer el HTML.
3. **Parseo de Nodos Quirúrgico (Para etiquetas no estándar):** Si la página es estática pero no usa la etiqueta `<table>`, utilizamos la librería **BeautifulSoup**. Esto nos permite buscar clases CSS específicas (ej. extraer todos los `<div class="precio">`) y construir el DataFrame manualmente fila por fila.

### ⚖️ Ética y Legalidad en el Web Scraping

**El Mito del "Acceso Público":**
El hecho de que una página web sea visible sin contraseña no convierte a sus datos en "Dominio Público" (Open Data). Los datos estructurados (tablas, catálogos, bases de datos subyacentes) suelen estar protegidos por derechos de autor y por los Términos de Servicio (ToS) del sitio.

**Auditoría de Cumplimiento (Compliance) antes de hacer Scraping:**
Como Ingenieros de Datos, antes de extraer información de terceros para uso comercial, debemos validar:
1. **Archivo `robots.txt`:** Verificar si el dominio permite el rastreo automatizado en la ruta deseada.
2. **Términos y Condiciones:** Leer explícitamente si el sitio prohíbe el uso de "bots", "scrapers" o "data mining".
3. **Privacidad de Datos:** Asegurar que no se están extrayendo PII (Personally Identifiable Information) que violen leyes locales (ej. Ley 25.326 en Argentina, GDPR en Europa).

**La Solución Corporativa:**
Si necesitamos datos de un competidor o proveedor, la ruta oficial es buscar si ofrecen una **API Pública y Oficial** (como hicimos con *Fruityvice* o como ofrece el Banco Central o AFIP). Las APIs están diseñadas precisamente para el consumo de máquinas bajo reglas claras.


##**Escribiendo archivos HTML**

In [19]:
top_peliculas.to_html('top_peliculas.html',index=False)

In [20]:
pd.read_html('top_peliculas.html')

[                            Film  Release year              Director  \
 0                   Citizen Kane          1941          Orson Welles   
 1                     Casablanca          1942        Michael Curtiz   
 2                  The Godfather          1972  Francis Ford Coppola   
 3             Gone with the Wind          1939        Victor Fleming   
 4             Lawrence of Arabia          1962            David Lean   
 ..                           ...           ...                   ...   
 95                 The Searchers          1956             John Ford   
 96              Bringing Up Baby          1938          Howard Hawks   
 97                    Unforgiven          1992        Clint Eastwood   
 98  Guess Who's Coming to Dinner          1967        Stanley Kramer   
 99           Yankee Doodle Dandy          1942        Michael Curtiz   
 
                       Production companies  Rank  
 0                       RKO Radio Pictures     1  
 1                

In [21]:
top_peliculas.to_csv('top_peliculas_1998', index=False)

In [22]:
datos = pd.read_csv('top_peliculas_1998')
datos.head()

,Film,Release year,Director,Production companies,Rank
0,Citizen Kane,1941,Orson Welles,RKO Radio Pictures,1
1,Casablanca,1942,Michael Curtiz,Warner Bros. Pictures,2
2,The Godfather,1972,Francis Ford Coppola,"Paramount Pictures, Alfran Productions",3
3,Gone with the Wind,1939,Victor Fleming,Selznick International Pictures,4
4,Lawrence of Arabia,1962,David Lean,Horizon Pictures,5


### 📤 Exportación Dual y Control de Calidad (QA)

**Concepto de Negocio (Multi-formato):**
Dependiendo de quién sea el consumidor final de los datos, debemos adaptar el formato de salida:
* **HTML (`to_html`):** Ideal para reportes estáticos y visuales (Front-end). Cualquier usuario de negocio puede abrirlo en su navegador sin conocimientos técnicos.
* **CSV (`to_csv`):** El estándar universal para la integración de sistemas (Back-end) y el almacenamiento ligero.

**Regla de Oro (Auditoría de Integridad):**
Nunca se asume que una operación de escritura (I/O) en el disco duro o en la nube fue exitosa solo porque no arrojó un error. Siempre debemos programar un bloque de lectura posterior para confirmar que los datos aterrizaron con su estructura, cantidad de filas y codificación intactas. En ambos casos, bloqueamos la exportación del índice (`index=False`) para evitar generar columnas numéricas basura.

In [18]:
import pandas as pd

# 1. Definición de Rutas (Entregables Finales)
RUTA_REPORTE_HTML = '/content/drive/MyDrive/Pandas/reporte_top_peliculas.html'
RUTA_BASE_CSV = '/content/drive/MyDrive/Pandas/top_peliculas_1998.csv'

# 2. Fase de Carga (Exportación Dual)
# Generamos el reporte visual para la Jefatura
df_peliculas.to_html(RUTA_REPORTE_HTML, index=False)

# Generamos la base cruda para el sistema
df_peliculas.to_csv(RUTA_BASE_CSV, index=False)

print("✅ Fase de Carga completada. Archivos generados físicamente.")

# -------------------------------------------------------------------
# 3. Auditoría de Calidad Automática (Test de Integridad QA)
# -------------------------------------------------------------------
print("\n--- Iniciando Auditoría de Datos ---")

# Volvemos a leer el CSV desde el disco duro
df_qa_csv = pd.read_csv(RUTA_BASE_CSV)

# Verificación lógica: ¿Tienen la misma cantidad de filas?
if len(df_qa_csv) == len(df_peliculas):
    print("🟢 Estado de Salud: APROBADO. Los datos no sufrieron pérdidas durante la escritura.")
else:
    print("🔴 ALERTA: Diferencia de registros detectada en la exportación.")

print(f"\nVista previa del archivo CSV consolidado ({len(df_qa_csv)} registros):")
display(df_qa_csv.head(3))

✅ Fase de Carga completada. Archivos generados físicamente.

--- Iniciando Auditoría de Datos ---
🟢 Estado de Salud: APROBADO. Los datos no sufrieron pérdidas durante la escritura.

Vista previa del archivo CSV consolidado (100 registros):


,Film,Release year,Director,Production companies,Rank
0,Citizen Kane,1941,Orson Welles,RKO Radio Pictures,1
1,Casablanca,1942,Michael Curtiz,Warner Bros. Pictures,2
2,The Godfather,1972,Francis Ford Coppola,"Paramount Pictures, Alfran Productions",3


##**Leyendo/escribiendo archivos XML**


In [14]:
import pandas as pd

In [15]:
datos_imdb = pd.read_xml('/content/drive/MyDrive/Pandas/imdb_top_1000.xml')

In [16]:
datos_imdb.head(3)

,index,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"


In [17]:
datos_imdb.to_xml('peliculas_imdb.xml')

### 🏗️ Exportación XML Corporativa (Estándar B2B)

**Concepto de Negocio:**
A diferencia de JSON o CSV, el formato XML es extremadamente estricto y se utiliza en entornos transaccionales regulados (como integraciones bancarias o sistemas fiscales). En estos ecosistemas, la estructura y el nombre de las etiquetas (tags) son obligatorios.

**Parámetros de Integración Senior:**
Al utilizar el método `to_xml()` de Pandas, debemos abandonar el comportamiento por defecto y utilizar dos parámetros clave para definir el esquema del documento:
* `root_name`: Define el nombre de la etiqueta principal que envuelve a todo el archivo (ej. `<Catalogo_IMDB>`).
* `row_name`: Define el nombre de la etiqueta que envuelve a cada registro individual (ej. `<Pelicula>`).

In [26]:
import pandas as pd

# 1. Definición de Rutas (Corregida con tu ruta exacta)
RUTA_XML_ORIGINAL = '/content/drive/MyDrive/Pandas/imdb_top_1000.xml'
RUTA_XML_EXPORTACION = '/content/drive/MyDrive/Pandas/catalogo_peliculas_b2b.xml'

# 2. Ingesta
df_imdb = pd.read_xml(RUTA_XML_ORIGINAL)
print(f"📥 Archivo original cargado. Total de registros: {len(df_imdb)}")

# 3. Transformación y Carga (Exportación parametrizada B2B)
# Inyectamos nuestra nomenclatura de negocio para reemplazar <data> y <row>
df_imdb.to_xml(
    RUTA_XML_EXPORTACION,
    index=False,
    root_name='Catalogo_IMDB',
    row_name='Pelicula'
)

print(f"✅ Exportación XML (Estándar Corporativo) exitosa en: {RUTA_XML_EXPORTACION}")

# -------------------------------------------------------------------
# 4. Auditoría de Calidad Automática (Test QA)
# -------------------------------------------------------------------
print("\n--- Iniciando Auditoría de Integridad XML ---")

# Pandas es lo suficientemente inteligente para detectar nuestras nuevas etiquetas al leer
df_qa_xml = pd.read_xml(RUTA_XML_EXPORTACION)

# Verificación lógica de registros
if len(df_qa_xml) == len(df_imdb):
    print("🟢 Estado de Salud: APROBADO. Las etiquetas personalizadas (<Catalogo_IMDB> y <Pelicula>) fueron asimiladas correctamente.")
else:
    print("🔴 ALERTA: Diferencia en la validación del esquema XML.")

# Resolvemos la redundancia promoviendo la columna 'index' a Clave Principal
df_qa_xml = df_qa_xml.set_index('index')

# Opcional (Clean Code): Le damos un nombre de negocio más claro a ese eje
df_qa_xml.index.name = 'ID_Registro'

# Mostramos el resultado limpio
display(df_qa_xml.head(2))


📥 Archivo original cargado. Total de registros: 1000
✅ Exportación XML (Estándar Corporativo) exitosa en: /content/drive/MyDrive/Pandas/catalogo_peliculas_b2b.xml

--- Iniciando Auditoría de Integridad XML ---
🟢 Estado de Salud: APROBADO. Las etiquetas personalizadas (<Catalogo_IMDB> y <Pelicula>) fueron asimiladas correctamente.


,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
ID_Registro,,,,,,,,,,,,,,,,
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"


### 📄 Anatomía del XML: El Estándar Corporativo

**Concepto de Negocio (Integraciones B2B):**
Mientras que JSON domina el desarrollo de aplicaciones web ágiles, XML es el estándar inamovible para transacciones financieras, gubernamentales (ej. AFIP) y sistemas heredados (Legacy). Su diseño jerárquico y estricto garantiza que no haya ambigüedades en auditorías.

**Estructura Jerárquica del Documento:**
1. **Declaración (`<?xml...?>`):** El encabezado técnico. Define la versión y la codificación (vital usar `UTF-8` para no corromper la letra "ñ" o los acentos en sistemas de facturación).
2. **Elemento Raíz (`<documento>`):** La caja contenedora maestra. Solo puede existir **uno** por archivo. Todo el resto de la información vive dentro de él.
3. **Elementos Secundarios (`<registro>`):** Son las "filas" de nuestra tabla de datos, anidadas dentro del elemento raíz.
4. **Atributos (`<registro id="01" estado="activo">`):** Metadatos incrustados directamente dentro de la etiqueta de apertura. Para Pandas, tanto los elementos secundarios anidados como los atributos se traducen en **columnas** dentro del DataFrame.